# Tutorial Variational Trajectory Planning: Build your first motion planner

This tutorial shows how you can plan trajectories by solving an optimization problem. We therefore use the CommonRoad scenario 'ZAM_Tutorial-1_2_T-1.xml'. 

## Task of the Ego Vehicle

The ego vehicle has the following specification in our scenario:
- obstacle type: car
- obstacle shape: rectangle with a width of 1.8m and a length of 5m
- initial state:
    - position: (15, 0.0) m
    - orientation: 0.00 rad
    - velocity: 22 m/s
- ID in ZAM_Tutorial-1_2_T-1.xml: 100

The following physical constraints apply to the ego vehicle:
- minimum acceleration: -6 m/s2
- maximum acceleration: 6 m/s2
- minimum jerk: -15 m/s3
- maximum jerk: 15 m/s3

The task of the ego vehicle is to reach its target velocity of 30.0 m/s without colliding with other obstacles. 


# Excercise 0: Read the created CommonRoad scenario

The CommonRoadFileReader reads the Scenario and the PlanningProblemSet from a CommonRoad XML-file. Please read the scenario from the CommonRoad file.

In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

# load the CommonRoad scenario that has been created in the CommonRoad tutorial
file_path = os.path.join(os.getcwd(), 'ZAM_Tutorial-1_2_T-1.xml')

scenario, planning_problem_set = 

# plot the scenario for each time step
for i in range(0, 40):
    plt.figure(figsize=(25, 10))
    draw_object(scenario, draw_params={'time_begin': i})
    draw_object(planning_problem_set)
    plt.gca().set_aspect('equal')
    plt.show()

# Helper Functions for visualization

The following functions are necessary for this tutorial.

In [ ]:
try:
    import matplotlib.pyplot as plt
    #%matplotlib inline
    #%config InlineBackend.figure_format = 'svg'
except ImportError:
    print('Matplotlib not installed. Please use pip(3) to install required package!')

try:
    import numpy as npy
except ImportError:
    print('Numpy not installed. Please use pip(3) to install required package!')
    
try:
    from cvxpy import *
except ImportError:
    print('CVXPy not installed. Please use pip(3) to install required package!')

class TIConstraints:
    a_min = -8
    a_max = 15
    s_min = 0
    s_max = 150
    v_min = 0
    v_max = 35
    j_min = -30
    j_max = 30
    

def plot_state_vector(x : Variable, c : TIConstraints, s_obj = None):
    plt.figure(figsize=(10,10))
    N = x.size[1]-1
    s_max = npy.maximum(150,100+0*npy.ceil(npy.amax(x[0,:].value.A.flatten())*1.1/10)*10)

    # Plot (x_t)_1.
    plt.subplot(4,1,1)
    x1 = x[0,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)),x1,'g')
    if s_obj is not None:
        plt.plot(npy.array(range(1,N+1)),s_obj[0],'b')
        plt.plot(npy.array(range(1,N+1)),s_obj[1],'r')
    plt.ylabel(r"$s$", fontsize=16)
    plt.yticks(npy.linspace(c.s_min, s_max, 3))
    plt.ylim([c.s_min, s_max])
    plt.xticks([])

    # Plot (x_t)_2.
    plt.subplot(4,1,2)
    x2 = x[1,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)),x2,'g')
    plt.yticks(npy.linspace(c.v_min,c.v_max,3))
    plt.ylim([c.v_min, c.v_max+2])
    plt.ylabel(r"$v$", fontsize=16)
    plt.xticks([])

    # Plot (x_t)_3.
    plt.subplot(4,1,3)
    x2 = x[2,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)),x2,'g')
    plt.yticks(npy.linspace(c.a_min,c.a_max,3))
    plt.ylim([c.a_min, c.a_max+2])
    plt.ylabel(r"$a$", fontsize=16)
    plt.xticks([])

    # Plot (x_t)_4.
    plt.subplot(4,1,4)
    x2 = x[3,:].value.A.flatten()
    plt.plot(npy.array(range(N+1)), x2,'g')
    plt.yticks(npy.linspace(c.j_min,c.j_max,3))
    plt.ylim([c.j_min-1, c.j_max+1])
    plt.ylabel(r"$j$", fontsize=16)
    plt.xticks(npy.arange(0,N+1,5))
    plt.xlabel(r"$k$", fontsize=16)
    plt.tight_layout()
    plt.show()

# Variables of the Optimization Problem

We use the convex optimization package cvxpy to formulate our optimization problem. The following code initializes all necessary variables such as the state and input vector as well as the cost matrices. 

In [ ]:
# problem data
N  = 40  # number of time steps
n  = 4   # length of state vector 
m  = 1   # length of input vector
dT = scenario.dt # time step


# set up variables
x = Variable(n,N+1) # optimization vector x contains n states per time step 
u = Variable(m,N) # optimization vector u contains 1 state

# set up constraints
c = TIConstraints()
c.a_min = -6 # Minimum feasible acceleration of vehicle
c.a_max = 6 # Maximum feasible acceleration of vehicle
c.s_min = 0 # Minimum allowed position
c.s_max = 100 # Maximum allowed position
c.v_min = 0 # Minimum allowed velocity (no driving backwards!)
c.v_max = 35 # Maximum allowed velocity (speed limit)
c.j_min = -15 # Minimum allowed jerk 
c.j_max = 15 # Maximum allowed jerk

# weights for cost function
w_s = 0
w_v = 8
w_a = 2
w_j = 2
Q = npy.eye(n)*npy.transpose(npy.array([w_s,w_v,w_a,w_j]))
w_u = 1
R = w_u

# Excercise 1: Consider the vehicle model 

In order to obtain kinematically feasible motion plans, the optimization problem needs to respect the constraints of the vehicle model. We use a linear vehicle model, which can be written in the following state-space form: $$x_{k+1} = A x_k + B u_k.$$ Please fill the matrices A and B according to the fourth-order point-mass model, discretized using the time step dT.

In [ ]:
A =
B = 

# Formulation of the Optimization Problem

We are now able to formulate the optimization problem. Please set the initial state of the ego vehicle with a velocity of 0 m/s and its reference velocity. Subsequently, we create the cost terms and constraints of the optimization problem.

In [ ]:
# get the initial state of the ego vehicle from the planning problem set
planning_problem = planning_problem_set.find_planning_problem_by_id(100)
initial_state = planning_problem.initial_state

# initial state of vehicle for the optimization problem (longitudinal position, velocity, acceleration, jerk)
x_0 = npy.array([[initial_state.position[0]],
                 [0],[0],[0]]) # initial state

# reference velocity
v_ref = 30.0
# Set up optimization problem
states = []
for k in range(N):
    # cost function
    cost = quad_form(x[:,k+1]-npy.transpose(npy.array([0,v_ref,0,0])),Q) + quad_form(u[:,k],R)
    # time variant state and input constraints
    constr = [x[:,k+1] == A*x[:,k] + B*u[:,k]]
    states.append( Problem(Minimize(cost), constr) )
    
# sums problem objectives and concatenates constraints.
prob = sum(states)
# initial state and goal state constraints
prob.constraints += [x[:,0] == x_0]

# Solve optimization problem
prob.solve(verbose=True)
print("Problem is convex: ",prob.is_dcp())
print("Problem solution is "+prob.status)

# plot results
plot_state_vector(x, TIConstraints())

# Exercise 2: Consider additional constraints

The obtained trajectory reaches the desired reference velocity of the ego vehicle. However, the physical constraints are not fulfilled. Please add the physical constraints to the optimization problem in order to obtain kinematically feasible trajectories. 

In [ ]:
tiConstraints = 
tiConstraints += 
tiConstraints += 

# Adjust problem
prob.constraints += tiConstraints

# Solve optimization problem
prob.solve()
print("Problem is convex: ",prob.is_dcp())
print("Problem solution is "+prob.status)

# plot results
plot_state_vector(x, c)


# Exercise 3: Extract the prediction of obstacles

In our scenario, the ego vehicle is surrounded by two other traffic participants -- one following and one leading vehicle. In order to not cause a collision with these two vehicles, we need to consider the positions of the two vehicles in our optimization problem. Please obtain the minimum and maximum feasible position of the ego vehicle for each point in time. Please note that each obstacle has a certain shape.   

In [ ]:
# extract obstacle from scenario
dyn_obstacles = scenario.dynamic_obstacles

# create constraints for minimum and maximum position
s_min = [] # minimum position constraint
s_max = [] # maximum position constraint

# go through obstacle list and distinguish between following and leading vehicle
for o in dyn_obstacles:
    
            
# plot vehicle motions
plt.plot(range(1,len(s_min)+1),s_min,'b')
plt.plot(range(1,len(s_max)+1),s_max,'r')

# Exercise 4: Plan the motion of the ego vehicle

The obtained collision constraint allow us to avoid collisions with the following and leading vehicle. Please add the constraints to our optimization problem and obtain a trajectory. 

In [ ]:
# initial state of ego vehicle
x_0 = npy.array([[initial_state.position[0]],
                 [initial_state.velocity],[0],[0]]) # initial state

# reference velocity
v_ref = 30.0

# Set up optimization problem
states = []
for k in range(N):
    # cost function
    cost = quad_form(x[:,k+1]-npy.transpose(npy.array([0,v_ref,0,0])),Q) + quad_form(u[:,k],R) 
    # single state and input constraints
    constr = [x[:,k+1] == A*x[:,k] + B*u[:,k]]
    # add obstacle constraint
    constr += 
    constr += 
    states.append( Problem(Minimize(cost), constr) )
    
# sums problem objectives and concatenates constraints.
prob = sum(states)
# constraints for all states and inputs
prob.constraints += tiConstraints
# initial state and goal state constraints
prob.constraints += [x[:,0] == x_0]

# Solve optimization problem
prob.solve(verbose=True)
print("Problem is convex:",prob.is_dcp())
print("Problem solution is "+prob.status)

# plot results
plot_state_vector(x, c, [s_min,s_max])

s_ego = x[0,:].value.A.flatten()
v_ego = x[1,:].value.A.flatten()

# Exercise 5

We have successfully planned the motion of the ego vehicle. Please convert the ego vehicle to a dynamic obstacle and visualize the scenario and the motion of the ego vehicle for each time step. 

In [ ]:
from commonroad.geometry.shape import Rectangle
from commonroad.scenario.obstacle import DynamicObstacle, ObstacleType
from commonroad.scenario.trajectory import Trajectory,State
from commonroad.prediction.prediction import TrajectoryPrediction

# generate state list of the ego vehicle's trajectory
state_list = [initial_state]
for i in range(1, 40):
    # compute new position
    # add new state to state_list

    
# create the planned trajectory starting at time step 1
ego_vehicle_trajectory = Trajectory(initial_time_step=1, state_list=state_list[1:])
# create the prediction using the planned trajectory and the shape of the ego vehicle
ego_vehicle_shape = Rectangle(length=5.0, width=1.8)
ego_vehicle_prediction = TrajectoryPrediction(trajectory=ego_vehicle_trajectory,
                                              shape=ego_vehicle_shape)

# the ego vehicle can be visualized by converting it into a DynamicObstacle
ego_vehicle_type = ObstacleType.CAR
ego_vehicle = DynamicObstacle(obstacle_id=100, obstacle_type=ego_vehicle_type,
                              obstacle_shape=ego_vehicle_shape, initial_state=initial_state,
                              prediction=ego_vehicle_prediction)

# plot the scenario and the ego vehicle for each time step
for i in range(0, 40):
    plt.figure(figsize=(25, 10))
    draw_object(scenario, draw_params={'time_begin': i})
    draw_object(ego_vehicle, draw_params={'time_begin': i, 'facecolor': 'g'})
    draw_object(planning_problem_set)
    plt.gca().set_aspect('equal')
    plt.show()